# Applications

# Multingual Search:

- because embedding produces vectors that convey meaning, vectors for the same phrase in different languages produces similar results. For example, "best chinese restaurants in downtown?", "市中心最好的中国餐厅" (the meanings are the same, but the embeddings are slightly different); "vacation spots in Miami", "迈阿密的度假胜地" share similar or even exact embeddings. (in this example, they are the same)

RAG:
- using a vector database as an external knowlegde base:
  - enable a large language model to leverage a vector database  as an external knowledge base of factual information.
- retrieve relevant info and provide to llm:
  - improve a llm by enabling it to retrieve relevant source material from the evctor database and read it as part of the prompt prior to generating an answer to the prompt.
- synergizes with a Vector Database
  - vector databases can be queried for concepts using natural langugae.
- Better to do RAG:
  - performing RAG is a lot more practical than having the LLM attend over its trained knowledge base
- Example: Visiting a library: (!)
  - it's akin to a human visiting a library and checking out and reading source material and books prior to writing a well thought out response to a question.

# Advantages of RAG:

list of the advantages of rag:
- reduce hallucinations - furthermore allow the user to identify hallucinations by comparig generated text to source text
- enable a llm to cite sources that it used to generate answers
- solve knowlegede intensive tasks and prompts more accurately.

# The RAG workflow:
- At its simplest the RAG workflow consists of 4 steps:
  - step 1: query a vector database
  - step 2: obtain relevant source objects
  - step 3: stuff the objects into the prompt
  - step 4: send the modified prompt to the llm to generate an answer.

and these 4 step rag workflow can be executed in Weaviate using one call.

In [ ]:
def json_print(data):
    print(json.dumps(data, indent=2))

In [ ]:
import weaviate, os, json
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

auth_config = weaviate.auth.AuthApiKey(api_key=os.getenv("WEAVIATE_API_KEY"))

client = weaviate.Client(
    url=os.getenv("WEAVIATE_API_URL"),
    auth_client_secret=auth_config,
    additional_headers={
        "X-Cohere-Api-Key": os.getenv("COHERE_API_KEY"),
        "X-Cohere-BaseURL": os.getenv("CO_API_URL")
    }
)

client.is_ready() #check if True

### 2. How many vectors are stored in this database

In [ ]:
print(json.dumps(client.query.aggregate("Wikipedia").with_meta_count().do(), indent=2))

### 3. Perform search over them to find concepts you are interested in!

In [ ]:
response = (client.query
            .get("Wikipedia",['text','title','url','views','lang'])
            .with_near_text({"concepts": "Vacation spots in california"})
            .with_limit(5)
            .do()
           )

json_print(response)

In [ ]:
response = (client.query
            .get("Wikipedia",['text','title','url','views','lang'])
            .with_near_text({"concepts": "Vacation spots in california"})
            .with_where({
                "path" : ['lang'],
                "operator" : "Equal",
                "valueString":'en'
            })
            .with_limit(3)
            .do()
           )

json_print(response)

In [ ]:
response = (client.query
            .get("Wikipedia",['text','title','url','views','lang'])
            .with_near_text({"concepts": "Miejsca na wakacje w Kalifornii"})
            .with_where({
                "path" : ['lang'],
                "operator" : "Equal",
                "valueString":'en'
            })
            .with_limit(3)
            .do()
           )

json_print(response)

In [ ]:
response = (client.query
            .get("Wikipedia",['text','title','url','views','lang'])
            .with_near_text({"concepts": "أماكن العطلات في كاليفورنيا"})
            .with_where({
                "path" : ['lang'],
                "operator" : "Equal",
                "valueString":'en'
            })
            .with_limit(3)
            .do()
           )

json_print(response)

## Retrieval Augmented Generation

### Single Prompt

In [ ]:
prompt = "Write me a facebook ad about {title} using information inside {text}"
result = (
  client.query
  .get("Wikipedia", ["title","text"])
  .with_generate(single_prompt=prompt)
  .with_near_text({
    "concepts": ["Vacation spots in california"]
  })
  .with_limit(3)
).do()

json_print(result)

### Group Task

In [ ]:
generate_prompt = "Summarize what these posts are about in two paragraphs."

result = (
  client.query
  .get("Wikipedia", ["title","text"])
  .with_generate(grouped_task=generate_prompt) # Pass in all objects at once
  .with_near_text({
    "concepts": ["Vacation spots in california"]
  })
  .with_limit(3)
).do()

json_print(result)
